In [1]:

from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.models import Sequential
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)

Num GPUs Available 1
2.10.1


In [3]:
class ResnetBlock(Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_1 = BatchNormalization()
        self.conv_2 = Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = BatchNormalization()
        self.merge = Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out


class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = BatchNormalization()
        self.pool_2 = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.avg_pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out

In [8]:
model = ResNet18(10)
model.build(input_shape = (None,100,100,3))
#use categorical_crossentropy since the label is one-hot encoded
from keras.optimizers import SGD
#model.load_weights('./saved_model/Resnet_18')
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 
model.summary()

Model: "res_net18_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          multiple                  9472      
                                                                 
 batch_normalization_20 (Bat  multiple                 256       
 chNormalization)                                                
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 resnet_block_8 (ResnetBlock  multiple                 74368     
 )                                                               
                                                                 
 resnet_block_9 (ResnetBlock  multiple                 74368     
 )                                                     

In [9]:
train_dir = "Dataset"
train_datagen = ImageDataGenerator(validation_split = 0.3)
#test_datagen = ImagedDataGenerator(1./255)

train_data = train_datagen.flow_from_directory(directory = train_dir, 
                                               target_size = (100,100),
                                               batch_size = 24,
                                               subset='training')
test_data = train_datagen.flow_from_directory(directory = train_dir, 
                                              target_size = (100,100),
                                              batch_size = 24,
                                              subset='validation')

Found 6180 images belonging to 10 classes.
Found 2640 images belonging to 10 classes.


In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

"""mc = ModelCheckpoint(filepath="./best_res_model.h5",
                     monitor = "val_accuracy",
                     verbose = 1,
                     save_best_only = "True")
"""
es = EarlyStopping(monitor = "val_accuracy",
                   min_delta = 0.01,
                   patience = 10,
                   verbose = 1)
cb = [es]

In [14]:

his = model.fit_generator(train_data,steps_per_epoch = 12 , epochs=100, validation_data= test_data, callbacks = cb)
model.save_weights('./saved_model/Resnet_18')

Epoch 1/100


C:\Users\amitp\AppData\Local\Temp\ipykernel_20260\3512904826.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  his = model.fit_generator(train_data,steps_per_epoch = 12 , epochs=100, validation_data= test_data, callbacks = cb)


12/12 [==============================] - 16s 1s/step - loss: 0.3977 - accuracy: 0.8854 - val_loss: 1.1783 - val_accuracy: 0.6977
Epoch 2/100
12/12 [==============================] - 16s 1s/step - loss: 0.3752 - accuracy: 0.8854 - val_loss: 0.7344 - val_accuracy: 0.7792
Epoch 3/100
12/12 [==============================] - 16s 1s/step - loss: 0.2771 - accuracy: 0.9236 - val_loss: 0.4636 - val_accuracy: 0.8553
Epoch 4/100
12/12 [==============================] - 16s 1s/step - loss: 0.2176 - accuracy: 0.9306 - val_loss: 0.4522 - val_accuracy: 0.8701
Epoch 5/100
12/12 [==============================] - 16s 1s/step - loss: 0.2596 - accuracy: 0.8993 - val_loss: 0.6078 - val_accuracy: 0.8409
Epoch 6/100
12/12 [==============================] - 16s 1s/step - loss: 0.2759 - accuracy: 0.8993 - val_loss: 1.0398 - val_accuracy: 0.7193
Epoch 7/100
12/12 [==============================] - 16s 1s/step - loss: 0.2868 - accuracy: 0.8889 - val_loss: 0.5887 - val_accuracy: 0.8193
Epoch 8/100
12/12 [======